In [2]:
#Sytem
import sys,os
import numpy as np
import pandas as pd
import warnings
import time
import random
import IPython
warnings.filterwarnings('ignore')

#Visualidation
import sweetviz as sv
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

#EDA
import sweetviz as sv
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import metrics


#model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from skopt import BayesSearchCV
import lightgbm as lgb



print('-'*40)

print("Python version: {}". format(sys.version))

print("pandas version: {}". format(pd.__version__))

print("matplotlib version: {}". format(matplotlib.__version__))

print("NumPy version: {}". format(np.__version__))


print("scikit-learn version: {}". format(sklearn.__version__))
print('-'*40)


----------------------------------------
Python version: 3.9.10 (main, Feb 22 2022, 13:54:07) 
[GCC 11.2.0]
pandas version: 1.3.5
matplotlib version: 3.5.1
NumPy version: 1.21.5
scikit-learn version: 1.1.1
----------------------------------------


In [3]:
Target = "Perished"
PATH_LOG = "./log.csv"
PATH_df = "./df"
ID = 00
SEED = 42

In [4]:
df_train = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")
log = pd.read_csv(PATH_LOG)
print('-'*25)
print("Shape of Train Data\n[row :{},column :{}]".format(
    df_train.shape[0], df_train.shape[1]))
print('-'*25)
print("Shape of Test Data\n[row :{},column :{}]".format(
    df_test.shape[0], df_test.shape[1]))
print('-'*25)

-------------------------
Shape of Train Data
[row :891,column :12]
-------------------------
Shape of Test Data
[row :418,column :11]
-------------------------


In [5]:
df_train["target"] = 0
df_test["target"] = 1
df_train = df_train.drop(columns=["Perished","PassengerId"])
df_test = df_test.drop(columns="PassengerId")

In [6]:
data = pd.concat([df_train,df_test])

In [7]:
data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,target
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [8]:
label = LabelEncoder()
ohe = OneHotEncoder()
data['Title'] = data['Name'].str.split(
        ", ", expand=True)[1].str.split(".", expand=True)[0]

title_names = (data.Title.value_counts() < 10)
data["Title"] = data["Title"].apply(lambda x: "other" if title_names.loc[x] == True else x)

In [9]:
faremean = data.Fare.mean()
data["Fare"].fillna(faremean,inplace=True)
data['FareBin'] = pd.qcut(data['Fare'], 10)
data['Fare_Code'] = label.fit_transform(data['FareBin'])

In [10]:
N_Age_Code = 3
data.replace({'Sex':{'male':0,'female':1}},inplace=True)
# data['Age_Code'] = label.fit_transform(pd.qcut(data['Age'], N_Age_Code))
data['Embarked_Code'] = label.fit_transform(data['Embarked'])

In [11]:
data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,target,Title,FareBin,Fare_Code,Embarked_Code
0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S,0,Mr,"(-0.001, 7.57]",0,2
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C,0,Mrs,"(41.579, 78.02]",8,0
2,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,Miss,"(7.854, 8.05]",2,2
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S,0,Mrs,"(41.579, 78.02]",8,2
4,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S,0,Mr,"(7.854, 8.05]",2,2


In [12]:
drop_columns = ["Name","Cabin","Embarked","Title","Ticket","FareBin"]
dataset = data.drop(columns=drop_columns)

In [13]:
dataset.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,target,Fare_Code,Embarked_Code
0,3,0,22.0,1,0,7.2500,0,0,2
1,1,1,38.0,1,0,71.2833,0,8,0
2,3,1,26.0,0,0,7.9250,0,2,2
3,1,1,35.0,1,0,53.1000,0,8,2
4,3,0,35.0,0,0,8.0500,0,2,2


In [14]:
X = dataset.drop(columns="target").values
y = dataset["target"].values

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
lgb_train = lgb.Dataset(X_train,y_train)
lgb_eval = lgb.Dataset(X_valid,y_valid,reference=lgb_train)
params = {
    "objective":"binary",
    'metric':'auc',
    'varbosity':-1
}
evals_result = {}

model = lgb.train(params,lgb_train,valid_sets = lgb_eval,
num_boost_round=1000,
early_stopping_rounds =100,evals_result = evals_result)

[LightGBM] [Warning] Unknown parameter: varbosity
[LightGBM] [Warning] Unknown parameter: varbosity
[LightGBM] [Info] Number of positive: 328, number of negative: 719
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000293 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 316
[LightGBM] [Info] Number of data points in the train set: 1047, number of used features: 8
[LightGBM] [Warning] Unknown parameter: varbosity
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.313276 -> initscore=-0.784848
[LightGBM] [Info] Start training from score -0.784848
[1]	valid_0's auc: 0.478359
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.481686
[3]	valid_0's auc: 0.508786
[4]	valid_0's auc: 0.517862
[5]	valid_0's auc: 0.512209
[6]	valid_0's auc: 0.525452
[7]	valid_0's auc: 0.514922
[8]	valid_0's auc: 0.515568
[9]	valid_0's auc: 0.517474
[10]	valid_0's auc: 0.518798
[11]	valid_0's au

In [28]:
tmetric = evals_result['train']['auc']
eval_metric = evals_result['eval']['auc']
tmetric[:5], eval_metric[:5]
plt.plot(tmetric, label='train auc')
plt.plot(eval_metric, label='eval auc')
plt.grid()
plt.legend()
plt.ylim(0, 1.1)

plt.xlabel('rounds')
plt.ylabel('auc')
plt.show()

KeyError: 'train'